In [2]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import pandas as pd

path= "/content/drive/MyDrive/phishing_urls/embeddings.pkl"
df = pd.read_pickle(path)
df

,URL,Label,Embeddings
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad,"[-0.08934366703033447, -0.010603426955640316, ..."
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad,"[-0.07558689266443253, 0.03260812163352966, 0...."
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad,"[-0.03573635593056679, 0.021130207926034927, -..."
3,mail.printakid.com/www.online.americanexpress....,bad,"[-0.05508413538336754, -0.03354843705892563, -..."
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad,"[-0.04461636021733284, 0.004012315534055233, 0..."
...,...,...,...
507190,23.227.196.215/,bad,"[-0.0011708217207342386, -0.018237954005599022..."
507191,apple-checker.org/,bad,"[-0.01951279491186142, -0.03594030812382698, 0..."
507192,apple-iclods.org/,bad,"[-0.046614840626716614, -0.014849048107862473,..."
507193,apple-uptoday.org/,bad,"[-0.043062400072813034, 0.005100815556943417, ..."


In [4]:
import numpy as np

# Convert string representations to actual lists
df['Embeddings'] = df['Embeddings'].apply(eval)

# Convert lists to numpy arrays
df['Embeddings'] = df['Embeddings'].apply(np.array)

# Check the type of the first element
print(type(df['Embeddings'][0]))

<class 'numpy.ndarray'>


In [5]:
import pandas as pd
import numpy as np

# Assuming 'df' is your DataFrame
urls_labels = df[['URL', 'Label']]
embeddings = np.stack(df['Embeddings'].values)

# Save URLs and labels to a CSV file
urls_labels.to_csv('/content/drive/MyDrive/phishing_urls/urls_labels.csv', index=False)


# Save embeddings to a .npy file
np.save('/content/drive/MyDrive/phishing_urls/embeddings.npy', embeddings)


Load back the df

In [11]:
# Load URLs and labels from a CSV file
urls_labels = pd.read_csv('/content/drive/MyDrive/phishing_urls/urls_labels.csv')

# Load embeddings from a .npy file
embeddings = np.load('/content/drive/MyDrive/phishing_urls/embeddings.npy')

# If you need to reconstruct the original DataFrame
df = urls_labels.copy()
df['Embeddings'] = list(embeddings)


In [12]:
len(df['Embeddings'][0])

384

In [13]:
# Convert labels to numerical values
df['Label'] = df['Label'].apply(lambda x: 1 if x == 'good' else 0)

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Assuming df_ is your DataFrame and it has the 'Embeddings' and 'Label' columns
X = np.stack(df['Embeddings'].values)  # Features (embeddings)
y = df['Label'].values  # Labels

# Assuming X and y are your feature and target arrays
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
# Use partial_fit for StandardScaler on large datasets
scaler = StandardScaler()
scaler.fit(X_train)  # Assuming you have enough memory to fit the scaler


StandardScaler()

In [37]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [38]:
mlp = MLPClassifier(
    hidden_layer_sizes=(192, 96),
    activation='relu',
    solver='adam',
    max_iter=15,
    random_state=42,
    verbose=True,
    early_stopping=True,  # Use early stopping
    n_iter_no_change=10,  # Number of iterations with no improvement to wait before stopping
    validation_fraction=0.2,  # Fraction of the training data to use as validation set for early stopping
)

# Fit the model
mlp.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = mlp.predict(X_test_scaled)
print(accuracy_score(y_test, y_pred))

Iteration 1, loss = 0.14538249
Validation score: 0.953926
Iteration 2, loss = 0.10284758
Validation score: 0.958609
Iteration 3, loss = 0.08386937
Validation score: 0.961110
Iteration 4, loss = 0.06971414
Validation score: 0.960987
Iteration 5, loss = 0.05797928
Validation score: 0.959915
Iteration 6, loss = 0.04806629
Validation score: 0.961554
Iteration 7, loss = 0.04139711
Validation score: 0.961172
Iteration 8, loss = 0.03357083
Validation score: 0.958806
Iteration 9, loss = 0.02974523
Validation score: 0.960260
Iteration 10, loss = 0.02532623
Validation score: 0.959286
Iteration 11, loss = 0.02299607
Validation score: 0.959878
Iteration 12, loss = 0.02086514
Validation score: 0.960740
Iteration 13, loss = 0.01909850
Validation score: 0.960383
Iteration 14, loss = 0.01731892
Validation score: 0.959114
Iteration 15, loss = 0.01668134
Validation score: 0.960260


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


0.9619081418389377


In [39]:
from sklearn.metrics import confusion_matrix
# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[20449  2285]
 [ 1579 77126]]


In [32]:
import torch
# Save the trained model to a .pth file
torch.save(mlp,'/content/drive/MyDrive/phishing_urls/mlpphishingmodel.pkl')